# This notenook for proposing metrics for accessing the conditions of our generated images

In [ ]:
import numpy
import os
import time
import numpy as np
import pandas as pd
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy import savez_compressed
from numpy.random import randint
from tensorflow.keras.models import load_model
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import LeakyReLU
from keras.layers import Dropout

In [ ]:
#Mount the Drive
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/deeplearning/")
!ls

In [0]:
#    INDICATE IF STARTING FRESH OR CONTINUING FROM PREVIOUS RUN
qRestart = True
if qRestart:
    epochs_done = 15
    epochs_goal = 100
else:
    epochs_done = 0
    epochs_goal = 100

We will define a classifier using our already made discriminator model. The difference is that now we will only use a sequential model and use a crossentropy loss function with softmax activation as we want our model to classify images belonging to one of the four categories.

In [0]:
#define the standalone discriminator model
def define_classifier(in_shape=(64,64,3)):
  model = Sequential()
  # normal
  model.add(Conv2D(128, (5,5), padding='same', input_shape=in_shape))
  model.add(LeakyReLU(alpha=0.2))
  # downsample to 32x32
  model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  # downsample to 16x16
  model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  # downsample to 8x8
  model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  # downsample to 4x4
  model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  # classifier
  model.add(Flatten())
  model.add(Dropout(0.4))
  model.add(Dense(4, activation='softmax'))
  # compile model
  opt = Adam(lr=0.0002, beta_1=0.5)
  model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
  model.summary()
  return model

We will load the real samples from our data for training purpose

In [0]:
# load and prepare training images
def load_real_samples():
  #Prepare the training Dataset
  real_data_file = "img_align_celeba_attractive_face.npz"
  # load the face dataset
  train_images = np.load(real_data_file)["arr_0"]
  # convert from unsigned ints to floats
  train_images = train_images.astype('float32')
  # scale from [0,255] to [-1,1]
  train_images = (train_images - 127.5) / 127.5
  print(train_images.shape)
  return train_images

Generate real samples for each batch of epoch of training. We will convert our categories to one hot encoding so that they can be use as desired output of our classifier

In [0]:
# select real samples
def generate_real_samples(dataset, n_samples):
  # choose random instances
  ix = randint(0, dataset.shape[0], n_samples)
  # retrieve selected images
  train_images = dataset[ix]
  # generate real class labels (1)
  df = pd.read_csv('results/assigned_categories.csv')
  labels = df["Category"].values
  train_labels = labels[ix]
  # print(ix)
  # print(train_labels)
  # convert integers to dummy variables (i.e. one hot encoded)
  encoded_train_labels = np_utils.to_categorical(train_labels, num_classes=4)
  #print(encoded_train_labels.shape)
  return train_images, encoded_train_labels

We will generate 10000 samples of our GAN Model to verify the accuracy of labelling of our generated images.

In [0]:
# load and prepare training images
def load_generated_samples():
  model_generator = load_model("results/models/generator_model_200.h5")
  CGAN_data_file = "results/latent_points/latent_points_10000.npz"
  latent_points = np.load(CGAN_data_file)["arr_0"]
  print(latent_points.shape)
  # generate images
  test_labels = randint(0, 4, 10000)
  print(test_labels)
  test_images = model_generator.predict([latent_points, test_labels])
  encoded_train_labels = np_utils.to_categorical(test_labels, num_classes=4)
  print(encoded_train_labels.shape)
  return test_images, encoded_train_labels 

In [0]:
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, c_model, dataset, n_samples=100):
  # prepare real samples
  X_real, y_real = generate_real_samples(dataset, n_samples)
  # evaluate discriminator on real examples
  _ , acc_real = c_model.evaluate(X_real, y_real, verbose=0)
  # summarize discriminator performance
  print('>Accuracy real: %.0f%%' % (acc_real*100))
  # save the generator model tile file
  filename = 'results/classifier_models/generator_model_classifier_%03d.h5' % (epoch+1)
  c_model.save(filename)

In [0]:
def restart(epochs_done):
    # gen_weights = array(model.get_weights())
    print("****  PULLING IN EPOCH: ", epochs_done)
    filename = 'results/classifier_models/generator_model_classifier_%03d.h5' % (epochs_done)
    c_model = load_model(filename, compile=True)
    c_model.trainable = True
    for layer in c_model.layers:
        layer.trainable = True
    c_model.summary()
    return c_model

In [0]:
#train the classifier
def train(c_model, dataset, epochs_done, epochs_goal, n_batch=128):
  bat_per_epo = int(dataset.shape[0] / n_batch)
  now = time.time()
  # manually enumerate epochs
  for i in range(epochs_done, epochs_goal):
  # enumerate batches over the training set
    for j in range(bat_per_epo):
      # get randomly selected 'real'samples
      X_real, y_real = generate_real_samples(dataset, n_batch) 
      # update discriminator model weights
      c_loss1, accuracy_t = c_model.train_on_batch(X_real, y_real)
      # summarize loss on this batch
      if (j+1) % 50==0:
        diff = int(time.time()-now)
        print('>Epoch: %d, Batch/Epoch:%d/%d, Model_loss: %.3f, Seconds: %d, Accuracy: %.0f%%' %(i+1, j+1, bat_per_epo, c_loss1, diff, (accuracy_t*100)))
  # evaluate the model performance, sometimes
    if (i+1) % 5 == 0:
      summarize_performance(i, c_model, dataset)

In [ ]:
if qRestart:
        c_model = restart(epochs_done = epochs_done)
else:
        # create the discriminator
        c_model = define_classifier()


# load image data
dataset = load_real_samples()
train(c_model, dataset, epochs_done=epochs_done, epochs_goal=epochs_goal, n_batch=128)

In [0]:
#Evaluation metric for generated images:
classifier_model = load_model(results/classifier_models/generator_model_classifier_100.h5)
#evaluate discriminator on generated examples
generated_images, generated_labels = load_generated_samples()
#summarize discriminator performance
loss, acc_gen = classifier_model.evaluate(generated_images, generated_labels, verbose=0)
#summarize discriminator performance


In [0]:
#summarize discriminator performance
print('>Accuracy Generated: %.0f%%' % (acc_gen*100))